# Setting up Pyspark

Installing Java

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Installing Apache Spark and Hadoop:

In [2]:
!wget https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz
!rm spark-3.5.3-bin-hadoop3.tgz   # Tidying up

--2025-03-17 22:05:35--  https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400864419 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.3-bin-hadoop3.tgz’

spark-3.5.3-bin-had 100%[===================>] 382.29M  21.6MB/s    in 18s     

2025-03-17 22:05:53 (20.9 MB/s) - ‘spark-3.5.3-bin-hadoop3.tgz’ saved [400864419/400864419]



Setting up environmental variables:

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"

Install the findpark

In [4]:
!pip install -q findspark
import findspark
findspark.init()

Create the entry point to Spark.

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").config("spark.driver.memory", "10g").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) #  This will format our output tables a bit nicer when not using the show() method
spark

Creating the SparkContext

In [6]:
sc = spark.sparkContext

In [7]:
# print(spark.sparkContext._conf.set("spark.driver.memory", '4g'))
# print(spark.sparkContext._conf.get("spark.driver.memory"))

# Importing our datasets

Books.csv

In [8]:
import kagglehub
path = kagglehub.dataset_download("trnnhtminh/goodread-books-ratings")
print("Path to dataset files:", path)
!mv {path}/merged_books_ratings.csv .
!mv merged_books_ratings.csv Books.csv
!ls

Path to dataset files: /root/.cache/kagglehub/datasets/trnnhtminh/goodread-books-ratings/versions/1
mv: cannot stat '/root/.cache/kagglehub/datasets/trnnhtminh/goodread-books-ratings/versions/1/merged_books_ratings.csv': No such file or directory
mv: cannot stat 'merged_books_ratings.csv': No such file or directory
Books.csv  Genres.csv  Ratings.csv  sample_data  spark-3.5.3-bin-hadoop3


Genres.csv. Because that file is created halfway through the data processing, it has to be uploaded manually. Below are two options: from google drive or a local directory.

In [9]:
# from google drive. If this doesn't work, check that the file has access set to "anyone with the link", and that the id (after /d/ in the link to the file) is correct
!gdown --id 15kIXt9psvX12bvFbzg8WST6k8WaO_pYt
!gdown --id 1PAQKjrUUei5DiOEqoRpJw3t0hoTFTtpf

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=15kIXt9psvX12bvFbzg8WST6k8WaO_pYt
To: /content/Genres.csv
100% 874k/874k [00:00<00:00, 80.3MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1PAQKjrUUei5DiOEqoRpJw3t0hoTFTtpf
From (redirected): https://drive.google.com/uc?id=1PAQKjrUUei5DiOEqoRpJw3t0hoTFTtpf&confirm=t&uuid=90d49cc7-1db5-46da-a6e8-b7970d75d14d
To: /content/Ratings.csv
100% 527M/527M [00:07<00:00, 73.8MB/s]


In [10]:
# from a local directory
# from google.colab import files
# uploaded = files.upload()

In [11]:
books = sc.textFile("Books.csv")
genres = sc.textFile("Genres.csv")
ratings = sc.textFile("Ratings.csv")

In [12]:
books.take(1)

['user_id,isbn,rating,authors,description,genres,pages,title,publishYear,publishMonth,publishDay']

In [13]:
genres.take(1)

['isbn,genre']

In [14]:
ratings.take(1)

['1,classic,3']

# MapReduce 1: Join

In [15]:
import time

In [23]:
join_rdd = books.join(genres) \
                .map(lambda pairs: pairs[1])
start = time.perf_counter()
join_output = join_rdd.collect()
end = time.perf_counter()
print("Execution time: " + (end - start))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 53 in stage 6.0 failed 1 times, most recent failure: Lost task 53.0 in stage 6.0 (TID 164) (fe445a9e0a60 executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:776)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at org.apache.spark.InterruptibleIterator.foreach(InterruptibleIterator.scala:28)
	at scala.collection.generic.Growable.$plus$plus$eq(Growable.scala:62)
	at scala.collection.generic.Growable.$pl

In [17]:
# for i in range(0, 20):
#   print(join_output[i])

# MapReduce 2: Group By

In [24]:
ratings_rdd = ratings.map(lambda row: row.split(",")) \
                     .map(lambda row: (row[1], row[2])) \
                     .reduceByKey(lambda x, y: int(x)+int(y))
start = time.perf_counter()
ratings_output = ratings_rdd.collect()
end = time.perf_counter()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/content/spark-3.5.3-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-3.5.3-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/spark-3.5.3-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o35.rdd

In [19]:
for i in ratings_output:
  print(i)

('fantasy', 9885284)
('horror', 1996196)
('philosophy', 1176248)
('young-adult', 7507825)
('psychology', 801236)
('manga', 116093)
('contemporary', 5370340)
('fiction', 19869613)
('historical-fiction', 5210228)
('romance', 6331677)
('christian', 443905)
('travel', 419692)
('graphic-novels', 539440)
('history', 1340160)
('cookbook', 69113)
('mystery', 4622843)
('book', 1070730)
('self-help', 554385)
('paranormal', 2588188)
('suspense', 2548798)
('poetry', 810084)
('thriller', 3376067)
('nonfiction', 2329726)
('gay-and-lesbian', 610)
('spirituality', 469554)
('music', 166195)
('religion', 750382)
('science-fiction', 3638978)
('chick-lit', 2117020)
('crime', 2705488)
('comic', 531088)
('biography', 1415946)
('business', 330201)
('science', 636070)
('humor-and-comedy', 3559)
('memoir', 1203974)
('art', 231543)
('classic', 8557100)
('sport', 115297)


In [20]:
print("Execution time: " + str(end - start))

Execution time: 89.56945022200125


In [21]:
print(sc._conf.get('spark.driver.memory'))

10g


# Stop Spark

In [22]:
# Stopping Spark:
# sc.stop()
# spark.stop()